This module combines both the scaled and non-scaled data for convienience use in subsequent steps. 

In [1]:
import os, sys
import pandas as pd 
import numpy as np
import requests 
from datetime import datetime, date, time, timedelta
import pytz
import matplotlib.pyplot as plt
import matplotlib.dates as mdate
from mpl_finance import candlestick_ohlc
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.ticker as mticker
import pickle
pd.set_option('display.max_columns', 999)
%load_ext autoreload
%matplotlib inline

In [88]:
#Get SOCIAL data
s = pd.read_pickle("social_data_slide_scaled_h.pkl")
s_cols = pd.read_pickle("col_dict_h.pkl")
s = s.drop('pumped_yn',axis=1)

In [89]:
#Check range of scaled data between 0 and 1
assert np.max(s[s.columns.drop(s_cols['core'])].max()) - 1 < 1e-6 
assert np.min(s[s.columns.drop(s_cols['core'])].min())  == 0

In [93]:
print(s.shape)
s.sample(4)

(59286, 1623)


,currency,exchange,timestamp,pumptime_edited,coin_name,coin_id,code_repo_stars_h0,code_repo_stars_h1,code_repo_stars_h2,code_repo_stars_h3,code_repo_stars_h4,code_repo_stars_h5,code_repo_stars_h6,code_repo_stars_h7,code_repo_stars_h8,code_repo_stars_h9,code_repo_stars_h10,code_repo_stars_h11,code_repo_stars_h12,code_repo_stars_h13,code_repo_stars_h14,code_repo_stars_h15,code_repo_stars_h16,code_repo_stars_h17,code_repo_stars_h18,code_repo_stars_h19,code_repo_stars_h20,code_repo_stars_h21,code_repo_stars_h22,code_repo_stars_h23,code_repo_stars_h24,code_repo_stars_h25,code_repo_stars_h26,code_repo_stars_h27,code_repo_stars_h28,code_repo_stars_h29,code_repo_stars_h30,code_repo_stars_h31,code_repo_stars_h32,code_repo_stars_h33,code_repo_stars_h34,code_repo_stars_h35,code_repo_stars_h36,code_repo_stars_h37,code_repo_stars_h38,code_repo_stars_h39,code_repo_stars_h40,code_repo_stars_h41,code_repo_stars_h42,code_repo_stars_h43,code_repo_stars_h44,code_repo_stars_h45,code_repo_stars_h46,code_repo_stars_h47,code_repo_stars_h48,code_repo_forks_h0,code_repo_forks_h1,code_repo_forks_h2,code_repo_forks_h3,code_repo_forks_h4,code_repo_forks_h5,code_repo_forks_h6,code_repo_forks_h7,code_repo_forks_h8,code_repo_forks_h9,code_repo_forks_h10,code_repo_forks_h11,code_repo_forks_h12,code_repo_forks_h13,code_repo_forks_h14,code_repo_forks_h15,code_repo_forks_h16,code_repo_forks_h17,code_repo_forks_h18,code_repo_forks_h19,code_repo_forks_h20,code_repo_forks_h21,code_repo_forks_h22,code_repo_forks_h23,code_repo_forks_h24,code_repo_forks_h25,code_repo_forks_h26,code_repo_forks_h27,code_repo_forks_h28,code_repo_forks_h29,code_repo_forks_h30,code_repo_forks_h31,code_repo_forks_h32,code_repo_forks_h33,code_repo_forks_h34,code_repo_forks_h35,code_repo_forks_h36,code_repo_forks_h37,code_repo_forks_h38,code_repo_forks_h39,code_repo_forks_h40,code_repo_forks_h41,code_repo_forks_h42,code_repo_forks_h43,code_repo_forks_h44,code_repo_forks_h45,code_repo_forks_h46,code_repo_forks_h47,code_repo_forks_h48,code_repo_subscribers_h0,code_repo_subscribers_h1,code_repo_subscribers_h2,code_repo_subscribers_h3,code_repo_subscribers_h4,code_repo_subscribers_h5,code_repo_subscribers_h6,code_repo_subscribers_h7,code_repo_subscribers_h8,code_repo_subscribers_h9,code_repo_subscribers_h10,code_repo_subscribers_h11,code_repo_subscribers_h12,code_repo_subscribers_h13,code_repo_subscribers_h14,code_repo_subscribers_h15,code_repo_subscribers_h16,code_repo_subscribers_h17,code_repo_subscribers_h18,code_repo_subscribers_h19,code_repo_subscribers_h20,code_repo_subscribers_h21,code_repo_subscribers_h22,code_repo_subscribers_h23,code_repo_subscribers_h24,code_repo_subscribers_h25,code_repo_subscribers_h26,code_repo_subscribers_h27,code_repo_subscribers_h28,code_repo_subscribers_h29,code_repo_subscribers_h30,code_repo_subscribers_h31,code_repo_subscribers_h32,code_repo_subscribers_h33,code_repo_subscribers_h34,code_repo_subscribers_h35,code_repo_subscribers_h36,code_repo_subscribers_h37,code_repo_subscribers_h38,code_repo_subscribers_h39,code_repo_subscribers_h40,code_repo_subscribers_h41,code_repo_subscribers_h42,code_repo_subscribers_h43,code_repo_subscribers_h44,code_repo_subscribers_h45,code_repo_subscribers_h46,code_repo_subscribers_h47,code_repo_subscribers_h48,code_repo_contributors_h0,code_repo_contributors_h1,code_repo_contributors_h2,code_repo_contributors_h3,code_repo_contributors_h4,code_repo_contributors_h5,code_repo_contributors_h6,code_repo_contributors_h7,code_repo_contributors_h8,code_repo_contributors_h9,code_repo_contributors_h10,code_repo_contributors_h11,code_repo_contributors_h12,code_repo_contributors_h13,code_repo_contributors_h14,code_repo_contributors_h15,code_repo_contributors_h16,code_repo_contributors_h17,code_repo_contributors_h18,code_repo_contributors_h19,code_repo_contributors_h20,code_repo_contributors_h21,code_repo_contributors_h22,code_repo_contributors_h23,code_repo_contributors_h24,code_repo_contributors_h25,code_repo_contributors_h26,code_repo_cont

In [28]:
#Get MARKET Data
m = pd.read_pickle("ohlcv_data_slide_scaled_h.pkl")
m_cols = pd.read_pickle("ohlcv_dict_h.pkl")


In [94]:
m.sample(3)
print(m.shape)

(59285, 314)


In [42]:
minmax_cols = list(m.filter(regex="min|max").columns)

In [43]:
#Check range of scaled data between 0 and 1
assert np.max(m[m.columns.drop(m_cols['core'] + minmax_cols)].max()) - 1 < 1e-6 
assert np.min(m[m.columns.drop(m_cols['core'])].min())  == 0

AssertionError: 

In [53]:
#There is an irregular point here where we are anomaly with scaling
t = m[m.columns.drop(m_cols['core'] + minmax_cols)].max()
t[t>2]

open_h0          2.390000e+02
high_h0          2.390000e+02
low_h0           2.360000e+02
close_h0         2.370000e+02
volumefrom_h0    7.959563e+14
volumeto_h0      1.896000e+09
dtype: float64

In [79]:
m[m.volumeto_h0 > 200]

,currency,exchange,pumptime_edited,coin_name,timestamp,event_date,event_time,open_h0,open_h1,open_h2,open_h3,open_h4,open_h5,open_h6,open_h7,open_h8,open_h9,open_h10,open_h11,open_h12,open_h13,open_h14,open_h15,open_h16,open_h17,open_h18,open_h19,open_h20,open_h21,open_h22,open_h23,open_h24,open_h25,open_h26,open_h27,open_h28,open_h29,open_h30,open_h31,open_h32,open_h33,open_h34,open_h35,open_h36,open_h37,open_h38,open_h39,open_h40,open_h41,open_h42,open_h43,open_h44,open_h45,open_h46,open_h47,open_h48,open_min,open_max,high_h0,high_h1,high_h2,high_h3,high_h4,high_h5,high_h6,high_h7,high_h8,high_h9,high_h10,high_h11,high_h12,high_h13,high_h14,high_h15,high_h16,high_h17,high_h18,high_h19,high_h20,high_h21,high_h22,high_h23,high_h24,high_h25,high_h26,high_h27,high_h28,high_h29,high_h30,high_h31,high_h32,high_h33,high_h34,high_h35,high_h36,high_h37,high_h38,high_h39,high_h40,high_h41,high_h42,high_h43,high_h44,high_h45,high_h46,high_h47,high_h48,high_min,high_max,low_h0,low_h1,low_h2,low_h3,low_h4,low_h5,low_h6,low_h7,low_h8,low_h9,low_h10,low_h11,low_h12,low_h13,low_h14,low_h15,low_h16,low_h17,low_h18,low_h19,low_h20,low_h21,low_h22,low_h23,low_h24,low_h25,low_h26,low_h27,low_h28,low_h29,low_h30,low_h31,low_h32,low_h33,low_h34,low_h35,low_h36,low_h37,low_h38,low_h39,low_h40,low_h41,low_h42,low_h43,low_h44,low_h45,low_h46,low_h47,low_h48,low_min,low_max,close_h0,close_h1,close_h2,close_h3,close_h4,close_h5,close_h6,close_h7,close_h8,close_h9,close_h10,close_h11,close_h12,close_h13,close_h14,close_h15,close_h16,close_h17,close_h18,close_h19,close_h20,close_h21,close_h22,close_h23,close_h24,close_h25,close_h26,close_h27,close_h28,close_h29,close_h30,close_h31,close_h32,close_h33,close_h34,close_h35,close_h36,close_h37,close_h38,close_h39,close_h40,close_h41,close_h42,close_h43,close_h44,close_h45,close_h46,close_h47,close_h48,close_min,close_max,volumefrom_h0,volumefrom_h1,volumefrom_h2,volumefrom_h3,volumefrom_h4,volumefrom_h5,volumefrom_h6,volumefrom_h7,volumefrom_h8,volumefrom_h9,volumefrom_h10,volumefrom_h11,volumefrom_h12,volumefrom_h13,volumefrom_h14,volumefrom_h15,volumefrom_h16,volumefrom_h17,volumefrom_h18,volumefrom_h19,volumefrom_h20,volumefrom_h21,volumefrom_h22,volumefrom_h23,volumefrom_h24,volumefrom_h25,volumefrom_h26,volumefrom_h27,volumefrom_h28,volumefrom_h29,volumefrom_h30,volumefrom_h31,volumefrom_h32,volumefrom_h33,volumefrom_h34,volumefrom_h35,volumefrom_h36,volumefrom_h37,volumefrom_h38,volumefrom_h39,volumefrom_h40,volumefrom_h41,volumefrom_h42,volumefrom_h43,volumefrom_h44,volumefrom_h45,volumefrom_h46,volumefrom_h47,volumefrom_h48,volumefrom_min,volumefrom_max,volumeto_h0,volumeto_h1,volumeto_h2,volumeto_h3,volumeto_h4,volumeto_h5,volumeto_h6,volumeto_h7,volumeto_h8,volumeto_h9,volumeto_h10,volumeto_h11,volumeto_h12,volumeto_h13,volumeto_h14,volumeto_h15,volumeto_h16,volumeto_h17,volumeto_h18,volumeto_h19,volumeto_h20,volumeto_h21,volumeto_h22,volumeto_h23,volumeto_h24,volumeto_h25,volumeto_h26,volumeto_h27,volumeto_h28,volumeto_h29,volumeto_h30,volumeto_h31,volumeto_h32,volumeto_h33,volumeto_h34,volumeto_h35,volumeto_h36,volumeto_h37,volumeto_h38,volumeto_h39,volumeto_h40,volumeto_h41,volumeto_h42,volumeto_h43,volumeto_h44,volumeto_h45,volumeto_h46,volumeto_h47,volumeto_h48,volumeto_min,volumeto_max,pumped_yn
37834,WPR,Binance,2019-06-03 11:14:37+00:00,ONE,1559560477,2019-06-03,11:14:37,239.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,239.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,236.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,237.0,0.0,0.

In [80]:
m[m.open_h0 > 200]

,currency,exchange,pumptime_edited,coin_name,timestamp,event_date,event_time,open_h0,open_h1,open_h2,open_h3,open_h4,open_h5,open_h6,open_h7,open_h8,open_h9,open_h10,open_h11,open_h12,open_h13,open_h14,open_h15,open_h16,open_h17,open_h18,open_h19,open_h20,open_h21,open_h22,open_h23,open_h24,open_h25,open_h26,open_h27,open_h28,open_h29,open_h30,open_h31,open_h32,open_h33,open_h34,open_h35,open_h36,open_h37,open_h38,open_h39,open_h40,open_h41,open_h42,open_h43,open_h44,open_h45,open_h46,open_h47,open_h48,open_min,open_max,high_h0,high_h1,high_h2,high_h3,high_h4,high_h5,high_h6,high_h7,high_h8,high_h9,high_h10,high_h11,high_h12,high_h13,high_h14,high_h15,high_h16,high_h17,high_h18,high_h19,high_h20,high_h21,high_h22,high_h23,high_h24,high_h25,high_h26,high_h27,high_h28,high_h29,high_h30,high_h31,high_h32,high_h33,high_h34,high_h35,high_h36,high_h37,high_h38,high_h39,high_h40,high_h41,high_h42,high_h43,high_h44,high_h45,high_h46,high_h47,high_h48,high_min,high_max,low_h0,low_h1,low_h2,low_h3,low_h4,low_h5,low_h6,low_h7,low_h8,low_h9,low_h10,low_h11,low_h12,low_h13,low_h14,low_h15,low_h16,low_h17,low_h18,low_h19,low_h20,low_h21,low_h22,low_h23,low_h24,low_h25,low_h26,low_h27,low_h28,low_h29,low_h30,low_h31,low_h32,low_h33,low_h34,low_h35,low_h36,low_h37,low_h38,low_h39,low_h40,low_h41,low_h42,low_h43,low_h44,low_h45,low_h46,low_h47,low_h48,low_min,low_max,close_h0,close_h1,close_h2,close_h3,close_h4,close_h5,close_h6,close_h7,close_h8,close_h9,close_h10,close_h11,close_h12,close_h13,close_h14,close_h15,close_h16,close_h17,close_h18,close_h19,close_h20,close_h21,close_h22,close_h23,close_h24,close_h25,close_h26,close_h27,close_h28,close_h29,close_h30,close_h31,close_h32,close_h33,close_h34,close_h35,close_h36,close_h37,close_h38,close_h39,close_h40,close_h41,close_h42,close_h43,close_h44,close_h45,close_h46,close_h47,close_h48,close_min,close_max,volumefrom_h0,volumefrom_h1,volumefrom_h2,volumefrom_h3,volumefrom_h4,volumefrom_h5,volumefrom_h6,volumefrom_h7,volumefrom_h8,volumefrom_h9,volumefrom_h10,volumefrom_h11,volumefrom_h12,volumefrom_h13,volumefrom_h14,volumefrom_h15,volumefrom_h16,volumefrom_h17,volumefrom_h18,volumefrom_h19,volumefrom_h20,volumefrom_h21,volumefrom_h22,volumefrom_h23,volumefrom_h24,volumefrom_h25,volumefrom_h26,volumefrom_h27,volumefrom_h28,volumefrom_h29,volumefrom_h30,volumefrom_h31,volumefrom_h32,volumefrom_h33,volumefrom_h34,volumefrom_h35,volumefrom_h36,volumefrom_h37,volumefrom_h38,volumefrom_h39,volumefrom_h40,volumefrom_h41,volumefrom_h42,volumefrom_h43,volumefrom_h44,volumefrom_h45,volumefrom_h46,volumefrom_h47,volumefrom_h48,volumefrom_min,volumefrom_max,volumeto_h0,volumeto_h1,volumeto_h2,volumeto_h3,volumeto_h4,volumeto_h5,volumeto_h6,volumeto_h7,volumeto_h8,volumeto_h9,volumeto_h10,volumeto_h11,volumeto_h12,volumeto_h13,volumeto_h14,volumeto_h15,volumeto_h16,volumeto_h17,volumeto_h18,volumeto_h19,volumeto_h20,volumeto_h21,volumeto_h22,volumeto_h23,volumeto_h24,volumeto_h25,volumeto_h26,volumeto_h27,volumeto_h28,volumeto_h29,volumeto_h30,volumeto_h31,volumeto_h32,volumeto_h33,volumeto_h34,volumeto_h35,volumeto_h36,volumeto_h37,volumeto_h38,volumeto_h39,volumeto_h40,volumeto_h41,volumeto_h42,volumeto_h43,volumeto_h44,volumeto_h45,volumeto_h46,volumeto_h47,volumeto_h48,volumeto_min,volumeto_max,pumped_yn
37834,WPR,Binance,2019-06-03 11:14:37+00:00,ONE,1559560477,2019-06-03,11:14:37,239.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,239.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,236.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,237.0,0.0,0.

...This coin is odd. Must eleminate this for hygiene checks...

In [83]:
assert m.shape[0] - m[m.open_h0 < 200].shape[0] == 1 #Make sure we deleted only 1 row

In [84]:
m = m[m.open_h0 < 200]

## MERGE

In [105]:
f = pd.merge(left=m,left_on=['currency','exchange','timestamp','coin_name','pumptime_edited']
        ,right=s,right_on=['currency','exchange','timestamp','coin_name','pumptime_edited']
         ,how='inner'
            )

In [106]:
f=f.drop_duplicates()

In [107]:
f.shape

(59285, 1932)

In [108]:
#save
f.to_pickle("final_data.pkl")